In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

#https://github.com/neural-maze/philoagents-course/blob/7792f5546048e7c109881fa17df4a9039a9d61b5/philoagents-api/src/philoagents/application/conversation_service/workflow/chains.py

import os
from pathlib import Path
import sys
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import logging

project_root = Path().resolve().parent  
sys.path.append(str(project_root))

load_dotenv()

from src.receipt_intelligence.config import settings
from src.receipt_intelligence.domain.prompts import Prompts




In [11]:
def get_chat_model(temperature: float = 0.7, model_name: str = settings.GROQ_LLM_MODEL) -> ChatGroq:
    return ChatGroq(
        api_key=os.getenv("GROQ_API_KEY"),
        model_name=model_name,
        temperature=temperature,
    )

def get_receiptbot_response_chain():
    model = get_chat_model()
    #model = model.bind_tools(tools)

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", Prompts.RECEIPT_BOT_PROMPT),
            MessagesPlaceholder(variable_name="messages"),
        ],
        template_format="jinja2",
    )

    return prompt | model

def get_conversation_summary_chain(summary: str = ""):

    model = get_chat_model()

    prompt = ChatPromptTemplate.from_messages(
        [
            MessagesPlaceholder(variable_name="messages"),
            ("human", Prompts.SUMMARY_PROMPT),
        ],
        template_format="jinja2",
    )

    return prompt | model


In [12]:
chain = get_receiptbot_response_chain()

response = chain.invoke({
    "messages": [("human", "What is the meaning of life?")]
})
print(response.content)

I can only help with questions about receipt information in the database. For example, you could ask me about the prices of certain products.


In [13]:
chain = get_receiptbot_response_chain()

response = chain.invoke({
    "messages": [("human", "What is the average price of bread?")]
})
print(response.content)

RECEIPT


In [ ]:
def classify_message(state: ConversationState):
    
    last_message = state.messages[-1]
    model = get_chat_model()

    prompt = [
            {
                "role": "system",
                "content": "Classify the user message as either: "
                "'generate_sqlquery_node': if the user asks a question directly related to receipt data"
                "'introduction_node': if the user asks about anything unrelated to the receipt data"
                "You can only output one of these: 'generate_sqlquery_node' or 'introduction_node'"
            },
            {
                "role": "user",
                "content": last_message.content
            }
        ]

    response = model.invoke(prompt)

    return response.content